In [1]:
import sys
sys.path.append('..')

from data_parsers import databento_file_parser
from simulator import SimpleSingleTickerSimulator

import pandas as pd
import math

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 80)

In [2]:
data_file_path = '../data/databento/AMZN/xnas-itch-20230508.mbp-10.dbn.zst'
amzn_df = databento_file_parser(data_file_path)
amzn_df

,ts_event,action,side,depth,price,size,flags,bid_px_00,ask_px_00,bid_sz_00,ask_sz_00,bid_px_01,ask_px_01,bid_sz_01,ask_sz_01,bid_px_02,ask_px_02,bid_sz_02,ask_sz_02,bid_px_03,ask_px_03,bid_sz_03,ask_sz_03,bid_px_04,ask_px_04,bid_sz_04,ask_sz_04,bid_px_05,ask_px_05,bid_sz_05,ask_sz_05,bid_px_06,ask_px_06,bid_sz_06,ask_sz_06,bid_px_07,ask_px_07,bid_sz_07,ask_sz_07,bid_px_08,ask_px_08,bid_sz_08,ask_sz_08,bid_px_09,ask_px_09,bid_sz_09,ask_sz_09,symbol
ts_recv,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-05-08 14:30:00.006929087,2023-05-08 14:30:00.006765120,C,N,0,105.30,100,130,105.30,105.31,600,100,105.29,105.32,479,608,105.28,105.33,953,1600,105.27,105.34,475,794,105.26,105.35,425,384,105.25,105.36,1275,951,105.24,105.37,600,875,105.23,105.38,814,283,105.22,105.39,172,1269,105.21,105.40,842,283,AMZN
2023-05-08 14:30:00.006942603,2023-05-08 14:30:00.006778449,C,N,1,105.29,100,130,105.30,105.31,600,100,105.29,105.32,379,608,105.28,105.33,953,1600,105.27,105.34,475,794,105.26,105.35,425,384,105.25,105.36,1275,951,105.24,105.37,600,875,105.23,105.38,814,283,105.22,105.39,172,1269,105.21,105.40,842,283,AMZN
2023-05-08 14:30:00.049863902,2023-05-08 14:30:00.049699564,A,N,0,105.31,100,130,105.30,105.31,600,200,105.29,105.32,379,608,105.28,105.33,953,1600,105.27,105.34,475,794,105.26,105.35,425,384,105.25,105.36,1275,951,105.24,105.37,600,875,105.23,105.38,814,283,105.22,105.39,172,1269,105.21,105.40,842,283,AMZN
2023-05-08 14:30:00.049868386,2023-05-08 14:30:00.049703780,C,N,0,105.30,100,130,105.30,105.31,500,200,105.29,105.32,379,608,105.28,105.33,953,1600,105.27,105.34,475,794,105.26,105.35,425,384,105.25,105.36,1275,951,105.24,105.37,600,875,105.23,105.38,814,283,105.22,105.39,172,1269,105.21,105.40,842,283,AMZN
2023-05-08 14:30:00.049890420,2023-05-08 14:30:00.049725052,C,N,2,105.28,100,130,105.30,105.31,500,200,105.29,105.32,379,608,105.28,105.33,853,1600,105.27,105.34,475,794,105.26,105.35,425,384,105.25,105.36,1275,951,105.24,105.37,600,875,105.23,105.38,814,283,105.22,105.39,172,1269,105.21,105.40,842,283,AMZN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-08 20:59:57.047924972,2023-05-08 20:59:57.047760707,C,N,0,105.67,6,130,105.65,105.68,2000,100,105.60,105.70,121,633,105.57,105.72,30,100,105.55,105.75,100,50,105.53,105.80,28,2,105.52,105.84,10,16,105.51,105.90,100,58,105.50,105.92,139,100,105.45,105.96,100,10,105.44,105.97,2,400,AMZN
2023-05-08 20:59:57.047958552,2023-05-08 20:59:57.047794809,A,N,0,105.66,100,130,105.66,105.68,100,100,105.65,105.70,2000,633,105.60,105.72,121,100,105.57,105.75,30,50,105.55,105.80,100,2,105.53,105.84,28,16,105.52,105.90,10,58,105.51,105.92,100,100,105.50,105.96,139,10,105.45,105.97,100,400,AMZN
2023-05-08 20:59:59.255118037,2023-05-08 20:59:59.254954329,A,N,0,105.66,100,130,105.66,105.68,200,100,105.65,105.70,2000,633,105.60,105.72,121,100,105.57,105.75,30,50,105.55,105.80,100,2,105.53,105.84,28,16,105.52,105.90,10,58,105.51,105.92,100,100,105.50,105.96,139,10,105.45,105.97,100,400,AMZN


In [3]:
# BOT 1

In [12]:
class BOT1(SimpleSingleTickerSimulator):
    """
    BOT1 is a market-making bot designed for a single ticker. It makes trading decisions 
    based on market conditions and inventory levels to optimize order placement and management.
    """

    def __init__(self, ticker, max_inventory, order_size_ratio=0.1):
        """
        Initializes the bot with specific trading parameters.

        :param ticker: The ticker symbol for trading.
        :param max_inventory: Maximum inventory level for risk management.
        :param order_size_ratio: Ratio to determine the size of the orders.
        """
        super().__init__(ticker)
        self.max_inventory = max_inventory
        self.order_size_ratio = order_size_ratio

    def run_algo(self, bid_orderbook, ask_orderbook, inventory, bid_orders, ask_orders):
        """
        Main algorithm function for making trading decisions.
        """
        self.manage_orders(bid_orderbook, ask_orderbook, bid_orders, ask_orders)
        self.adjust_orders_based_on_market(bid_orderbook, ask_orderbook, inventory, bid_orders, ask_orders)

    def manage_orders(self, bid_orderbook, ask_orderbook, bid_orders, ask_orders):
        """
        Manages existing orders, canceling those that are outside the top levels of the orderbook.
        """
        self.cancel_orders_outside_top_levels(bid_orderbook, bid_orders, 'BID')
        self.cancel_orders_outside_top_levels(ask_orderbook, ask_orders, 'ASK')

    def cancel_orders_outside_top_levels(self, orderbook, orders, side):
        """
        Cancels orders that are outside the top levels of the order book.
        """
        for price in list(orders.keys()):
            if not self.is_order_within_top_levels(price, orderbook, side):
                self.cancel_order(price, side=side)

    def is_order_within_top_levels(self, price, orderbook, side):
        """
        Checks if an order is within the top levels of the order book.
        """
        
        if side == 'ASK':
            top_level_price = orderbook[0][0]
            bottom_level_price = orderbook[self.NO_OF_ORDERBOOK_LEVELS - 1][0]
            return top_level_price <= price <= bottom_level_price
        elif side == 'BID':
            top_level_price = orderbook[0][0]
            bottom_level_price = orderbook[self.NO_OF_ORDERBOOK_LEVELS - 1][0]
            return top_level_price >= price >= bottom_level_price
        else:
            raise ValueError('Invalid Side??')

    def place_or_adjust_order(self, orderbook, orders, inventory, side, target_price):
        """
        Places or adjusts an order at a target price.
        """
        target_price = round(target_price,  2)
        if len(orders) == 0:
            if self.is_order_within_top_levels(target_price, orderbook, side):
                size = self.calculate_order_size(orderbook, inventory, side)
                self.place_order(target_price, size, side=side)

        elif len(orders) == 1:
            order_price = list(orders.keys())[0]
            if abs(order_price - target_price) >= 0.02:
                self.cancel_order(order_price, side=side)
                if self.is_order_within_top_levels(target_price, orderbook, side):
                    size = self.calculate_order_size(orderbook, inventory, side)
                    self.place_order(target_price, size, side=side)
                
    def calculate_order_size(self, orderbook, inventory, side):
        """
        Calculates order size dynamically based on market depth and inventory.
        """
        market_depth_size = orderbook[1][1]  # Using level 2 for size reference
        return int(market_depth_size * self.order_size_ratio)

    def adjust_orders_based_on_market(self, bid_orderbook, ask_orderbook, inventory, bid_orders, ask_orders):
        """
        Adjusts the orders based on the current market conditions.
        """
        bid_price, ask_price = self.calculate_bid_ask_price(bid_orderbook, ask_orderbook, inventory)
        self.place_or_adjust_order(bid_orderbook, bid_orders, inventory, 'BID', bid_price)
        self.place_or_adjust_order(ask_orderbook, ask_orders, inventory, 'ASK', ask_price)

    def calculate_bid_ask_price(self, bid_orderbook, ask_orderbook, inventory):
        """
        Compute the bid and ask prices based on orderbook status and inventory
        Needs to be overwritten by the model
        """
        spread = ask_orderbook[0][0] - bid_orderbook[0][0]
        mid_price = (ask_orderbook[0][0] + bid_orderbook[0][0])/2
        return mid_price - spread, mid_price + spread

# Instantiate and run the bot
bot1 = BOT1(ticker='AMZN', max_inventory=1000, order_size_ratio=0.1)
bot1.run_sim(amzn_df)

self.current_ts=Timestamp('2023-05-08 14:36:52.521881676') 
self.BID_SIM_ORDER_BOOK={0: (105.23, 105), 1: (105.22, 754), 2: (105.21, 375), 3: (105.2, 1575), 4: (105.19, 303), 5: (105.18, 1461), 6: (105.17, 10279), 7: (105.16, 1661), 8: (105.15, 225), 9: (105.14, 2775)} 
self.ASK_SIM_ORDER_BOOK={0: (105.24, 358), 1: (105.25, 800), 2: (105.26, 600), 3: (105.27, 752), 4: (105.28, 975), 5: (105.29, 696), 6: (105.3, 225), 7: (105.31, 725), 8: (105.32, 761), 9: (105.33, 625)} 
self.BID_ALGO_ORDERS={105.22: 47} 
self.ASK_ALGO_ORDERS={105.24: 139} 
self.ALGO_POSITION=1423 

self.current_ts=Timestamp('2023-05-08 14:44:22.246361568') 
self.BID_SIM_ORDER_BOOK={0: (105.36, 300), 1: (105.35, 638), 2: (105.34, 275), 3: (105.33, 574), 4: (105.32, 375), 5: (105.31, 229), 6: (105.3, 38996), 7: (105.29, 276), 8: (105.28, 510), 9: (105.27, 376)} 
self.ASK_SIM_ORDER_BOOK={0: (105.37, 911), 1: (105.38, 700), 2: (105.39, 939), 3: (105.4, 409), 4: (105.41, 1325), 5: (105.42, 675), 6: (105.43, 875), 7: (105.4